In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
data = pd.read_csv("../data/derived/CarCrashData-Cleaned.csv")
data.datetime = pd.to_datetime(data.datetime, format="%Y/%m/%d %H:%M:%S")

In [3]:
data.head()

,datetime,day_of_week,lga,latitude,longitude,type,road_class,is_hit_and_run,light_condition,police_attended,...,total_non_injury,total_male,total_female,total_bicyclist,totaL_passenger,total_driver,total_pedestrian,total_pillion,total_motorist,total_unknown
0,2015-07-01 01:37:00,Wednesday,Wyndham,-37.868424,144.733170,Overturned (no collision),Arterial Other,False,Dark (lights on),True,...,0,1,0,0,0,0,0,0,1,0
1,2015-07-01 03:05:00,Wednesday,Monash,-37.893493,145.093413,Vehicle,Arterial Highway,False,Dark (lights on),True,...,2,2,0,0,1,2,0,0,1,0
2,2015-07-01 04:10:00,Wednesday,Yarra Ranges,-37.897932,145.357961,Animal,Arterial Other,False,Dark (no lights),False,...,0,1,0,0,0,0,0,0,1,0
3,2015-07-01 04:30:00,Wednesday,Geelong,-38.150540,144.581600,Vehicle,NaN,False,Dark (no lights),True,...,1,1,2,0,0,1,2,0,0,0
4,2015-07-01 04:30:00,Wednesday,Casey,-37.996330,145.269155,Pedestrian,Freeway,False,Dark (lights on),True,...,1,2,0,0,0,1,1,0,0,0


In [4]:
data['year'] = data.datetime.dt.year

In [5]:
df1 = data.groupby(["lga", "year"]).sum().drop(columns=["latitude", "longitude"])

In [6]:
df2 = data.groupby(["lga", "year"]).count().drop(columns=['day_of_week',
 'latitude',
 'longitude',
 'type',
 'road_class',
 'is_hit_and_run',
 'light_condition',
 'police_attended',
 'road_geometry',
 'speed_zone',
 'is_alcohol_related',
 'is_alcohol_time',
 'severity',
 'total_persons',
 'total_fatal',
 'total_serious_injury',
 'total_other_injury',
 'total_non_injury',
 'total_male',
 'total_female',
 'total_bicyclist',
 'totaL_passenger',
 'total_driver',
 'total_pedestrian',
 'total_pillion',
 'total_motorist',
 'total_unknown']
).reindex().rename(columns={"datetime": "total_crashes"})

In [54]:
pd.merge(df1, df2, on=["lga", "year"]).to_csv("../data/derived/CarCrashData-Summarised.csv")

In [60]:
df = data.groupby(["lga", data.datetime.dt.year, data.datetime.dt.month, "speed_zone"]).count().iloc[:,:1]
df.head()

datetime
lga    datetime datetime speed_zone          
Alpine 2015     7        100 km/hr          2
                8        100 km/hr          1
                         50 km/hr           1
                         60 km/hr           1
                         Off road           1

In [61]:
df.index.names = ["lga", "year", "month", "speed_zone"]
df.rename(columns={"datetime": "total_crashes"}, inplace=True)
df = df.reset_index()

In [63]:
df["datetime"] = pd.to_datetime(df.year.astype(str) + "-" + df.month.astype(str).str.rjust(2, "0") + "-01", format="%Y-%m-%d")
df.head()

,lga,year,month,speed_zone,total_crashes,datetime
0,Alpine,2015,7,100 km/hr,2,2015-07-01
1,Alpine,2015,8,100 km/hr,1,2015-08-01
2,Alpine,2015,8,50 km/hr,1,2015-08-01
3,Alpine,2015,8,60 km/hr,1,2015-08-01
4,Alpine,2015,8,Off road,1,2015-08-01


In [58]:
df.to_csv("../data/derived/temp_data_2.csv", index=False)